# Projeto 1 - Ciência dos Dados

Nome: Vinícius de Barros Gomes

Nome: Gabriel Nakano Fujihara

Nome: Pedro Roberto Fernandes Noronha

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from unidecode import unidecode
from sklearn.model_selection import train_test_split

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\dev\DataScienceProject


Carregando a base de dados com os tweets classificados manualmente:

In [4]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)


,Mensagem,"Target (Loja e Sistema, Entrega,Conteudo)"
0,"Não li, até pq não consegui baixar. Vcs podem ...",Loja e Sistema
1,Não tem nada indicando que é em português de P...,Loja e Sistema
2,"Produto horrível, veio com várias páginas em b...",Entrega
3,O minimo é não ler esse livro,Conteúdo do Livro
4,Parece que o autor não leu o próprio livro!! 🙄,Conteúdo do Livro


In [5]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,"Target (Loja e Sistema, Entrega,Conteudo)"
0,"Apesar da narrativa prender o leitor, os argum...",Conteúdo do Livro
1,A formatação desse ebook atrapalha muito a lei...,Loja e Sistema
2,Não é o primeiro livro da Record que oxida em ...,Entrega
3,O hype que envolve esse livro é ENORME então e...,Conteúdo do Livro
4,"meu deus, essa escritora conseguiu estragar um...",Conteúdo do Livro


___
## Classificador automático



Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Buscamos avaliar se os livros oferecidos pela amazon são ou não bons, portanto, foi ignorado quando haviam críticas exclusivamente ao serviço da amazon, como entrega e ou qualidade física do produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
#limpando a pontuação, acentos e deixando o texto minúsculo
def cleanup(text):
    text=text.lower()
    punctuation = '[´"!-.:?;$''<>]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def remover_acentos(texto):
    texto_sem_acentos = unidecode(texto)
    return texto_sem_acentos


In [7]:
#Limpa o df
def df0(df):
    df['Mensagem'] = df['Mensagem'].apply(cleanup)
    df['Mensagem'] = df['Mensagem'].apply(remover_acentos)
    novo_df = pd.concat([df[['Mensagem']], df[['Target (Loja e Sistema, Entrega,Conteudo)']]], axis=1)
    return novo_df
train=df0(train)
train['Mensagem']=train['Mensagem'].astype('category')

In [8]:
df_Loja_Sistema=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Loja e Sistema']
df_Entrega=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Entrega']
df_Conteudo=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Conteúdo do Livro']
quant_loja=len(df_Loja_Sistema)
quant_Conteudo=len(df_Conteudo)
quant_Entrega=len(df_Entrega)


In [9]:
#Conta a quantidade de palavras dentro de um dataframe
def ll(df):
    todas_as_frases=df["Mensagem"]
    todas_as_palavras = []
    for i in todas_as_frases:
        frase=[]
        x=i.split(" ")
        for u in x:
            if u not in frase:
                frase.append(u)
        for d in frase:
            todas_as_palavras.append(d)
    todas_as_palavrasSeries=pd.Series(todas_as_palavras)
    return todas_as_palavrasSeries.value_counts()
quant_palavras_Entrega= ll(df_Entrega)
quant_palavras_Conteudo= ll(df_Conteudo)
quant_palavras_Loja= ll(df_Loja_Sistema)

In [10]:
quantidade_total=quant_loja + quant_Conteudo + quant_Entrega
def laplace(x,q):
    return (x+1)/(q+quantidade_total)

In [11]:
#Classificando o dataframe teste com o classificador
def classifica(df):
    df_Loja_Sistema=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Loja e Sistema']
    df_Entrega=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Entrega']
    df_Conteudo=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Conteúdo do Livro']
    quant_loja=len(df_Loja_Sistema)
    quant_Conteudo=len(df_Conteudo)
    quant_Entrega=len(df_Entrega)
    quant_palavras_Entrega= ll(df_Entrega)
    quant_palavras_Conteudo= ll(df_Conteudo)
    quant_palavras_Loja= ll(df_Loja_Sistema)
    df['Mensagem']=df['Mensagem'].astype('category')
    coluna_de_resposta=[]
    for frase in df["Mensagem"]:
        frase=cleanup(frase)
        frase=remover_acentos(frase)
        frase=frase.lower()
        palavras=frase.split(' ')
        prob_Loja_Sistema=1
        prob_Entrega=1
        prob_Conteudo=1
        for palavra in palavras:
            if palavra in quant_palavras_Loja:
                prob_Loja_Sistema*=(quant_palavras_Loja[palavra]/quant_loja)
            else:
                prob_Loja_Sistema*=laplace(0,quant_loja)
        for palavra in palavras:
            if palavra in quant_palavras_Entrega:
                prob_Entrega*=(quant_palavras_Entrega[palavra]/quant_Entrega)
            else:
                prob_Entrega*=laplace(0,quant_Entrega)
        for palavra in palavras:
            if palavra in quant_palavras_Conteudo:
                prob_Conteudo*=(quant_palavras_Conteudo[palavra]/quant_Conteudo)
            else:
                prob_Conteudo*=laplace(0,quant_Conteudo)
        maior_prob=max(prob_Conteudo , prob_Entrega , prob_Loja_Sistema)
        if maior_prob==prob_Conteudo:
            coluna_de_resposta.append('Conteúdo do Livro')
        elif maior_prob==prob_Entrega:
            coluna_de_resposta.append("Entrega")
        elif maior_prob==prob_Loja_Sistema:
            coluna_de_resposta.append("Loja e Sistema")
    coluna_de_resposta_Series=pd.Series(coluna_de_resposta)
    df['Respostas']=coluna_de_resposta_Series
    novo_df = pd.concat([df['Mensagem'],df['Respostas']], axis=1)
    return novo_df

classifica(test)

,Mensagem,Respostas
0,"Apesar da narrativa prender o leitor, os argum...",Conteúdo do Livro
1,A formatação desse ebook atrapalha muito a lei...,Loja e Sistema
2,Não é o primeiro livro da Record que oxida em ...,Conteúdo do Livro
3,O hype que envolve esse livro é ENORME então e...,Conteúdo do Livro
4,"meu deus, essa escritora conseguiu estragar um...",Conteúdo do Livro
...,...,...
245,"<a class=""a-link-normal"" data-hook=""product-li...",Entrega
246,O livro tem muitas coisas boas sobre o assunto...,Conteúdo do Livro
247,Infelizmente a história é fraca e a escrita de...,Conteúdo do Livro
248,Ao longo do livro me senti extremamente mal ao...,Conteúdo do Livro


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [12]:
#verificando a acurácia do classificador a partir do arquivo teste
def acuracia(df):
    df_respostas=classifica(df)
    novo_df=pd.concat([df["Target (Loja e Sistema, Entrega,Conteudo)"],df_respostas['Respostas']], axis=1)
    acertos=0
    for i in novo_df.index:
        if novo_df["Target (Loja e Sistema, Entrega,Conteudo)"][i]==novo_df["Respostas"][i]:
            acertos+=1
    quantidade_respostas=len(df_respostas["Respostas"])
    return "Acurácia é: {0}%".format(acertos*100/quantidade_respostas)
print(acuracia(test))

Acurácia é: 86.8%


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [35]:
train_test=train.merge(test,how="outer")
train_test=train_test.drop(columns="Respostas")
train_test
#Embaralhando o dataframe em conjunto para separar em 2
def embaralha(df):
    train, test = train_test_split(df, test_size=0.4)
    return(train,test)
x=embaralha(train_test)
y=x[0]
z=x[1]

print(y)

                                              Mensagem  \
693  fuja desse lixo e um desperdicio de tempo inte...   
902  O livro é muito bom mas a edição é péssima! Fo...   
147  ganhei na compra do kindle nao e estilo de liv...   
231  incapaz de conviver com ideias diferentes na a...   
529  traducao muito muito ruim conteudo faltando al...   
..                                                 ...   
213  muita giria muitas vezes fiquei sem entender d...   
60   uma pena a editora ter se prestado a esse dess...   
647  apesar das criticas e sucesso nao consegui ter...   
75   o pior livro do autor que ja li chato enrolado...   
274  estava com saudades de todos os personagens ma...   

    Target (Loja e Sistema, Entrega,Conteudo)  
693                         Conteúdo do Livro  
902                            Loja e Sistema  
147                            Loja e Sistema  
231                         Conteúdo do Livro  
529                         Conteúdo do Livro  
..             

In [34]:
#adaptando as funções anteriores para a divisão do dataframe conjunto
def classifica(df,dfp):
    df_Loja_Sistema=train.loc[dfp["Target (Loja e Sistema, Entrega,Conteudo)"]=='Loja e Sistema']
    df_Entrega=train.loc[dfp["Target (Loja e Sistema, Entrega,Conteudo)"]=='Entrega']
    df_Conteudo=train.loc[dfp["Target (Loja e Sistema, Entrega,Conteudo)"]=='Conteúdo do Livro']
    quant_loja=len(df_Loja_Sistema)
    quant_Conteudo=len(df_Conteudo)
    quant_Entrega=len(df_Entrega)
    quant_palavras_Entrega= ll(df_Entrega)
    quant_palavras_Conteudo= ll(df_Conteudo)
    quant_palavras_Loja= ll(df_Loja_Sistema)
    df['Mensagem']=df['Mensagem'].astype('category')
    coluna_de_resposta=[]
    for frase in df["Mensagem"]:
        frase=cleanup(frase)
        frase=remover_acentos(frase)
        frase=frase.lower()
        palavras=frase.split(' ')
        prob_Loja_Sistema=1
        prob_Entrega=1
        prob_Conteudo=1
        for palavra in palavras:
            if palavra in quant_palavras_Loja:
                prob_Loja_Sistema*=(quant_palavras_Loja[palavra]/quant_loja)
            else:
                prob_Loja_Sistema*=laplace(0,quant_loja)
        for palavra in palavras:
            if palavra in quant_palavras_Entrega:
                prob_Entrega*=(quant_palavras_Entrega[palavra]/quant_Entrega)
            else:
                prob_Entrega*=laplace(0,quant_Entrega)
        for palavra in palavras:
            if palavra in quant_palavras_Conteudo:
                prob_Conteudo*=(quant_palavras_Conteudo[palavra]/quant_Conteudo)
            else:
                prob_Conteudo*=laplace(0,quant_Conteudo)
        maior_prob=max(prob_Conteudo , prob_Entrega , prob_Loja_Sistema)
        if maior_prob==prob_Conteudo:
            coluna_de_resposta.append('Conteúdo do Livro')
        elif maior_prob==prob_Entrega:
            coluna_de_resposta.append("Entrega")
        elif maior_prob==prob_Loja_Sistema:
            coluna_de_resposta.append("Loja e Sistema")
    coluna_de_resposta_Series=pd.Series(coluna_de_resposta)
    df['Respostas']=coluna_de_resposta_Series
    novo_df = pd.concat([df['Mensagem'],df['Respostas']], axis=1)
    return novo_df
def acuracia(df,dfp):
    df_respostas=classifica(df,dfp)
    novo_df=pd.concat([df["Target (Loja e Sistema, Entrega,Conteudo)"],df_respostas['Respostas']], axis=1)
    acertos=0
    for i in novo_df.index:
        if novo_df["Target (Loja e Sistema, Entrega,Conteudo)"][i]==novo_df["Respostas"][i]:
            acertos+=1
    quantidade_respostas=len(df_respostas["Respostas"])
    return acertos*100/quantidade_respostas
acurracy_list=[]
for i in range(100):
    x=embaralha(train_test)
    y=x[0]
    z=x[1]
    p=classifica(z,y)
    acurracy_list.append(p)
print(acurracy_list)


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**